In [5]:
import paho.mqtt.client as PahoMQTT

class MyMQTT:
	def __init__(self, clientID, broker, port, notifier):
		self.broker = broker
		self.port = port
		self.notifier = notifier
		self.clientID = clientID

		self._topic = ""
		self._isSubscriber = False

		# create an instance of paho.mqtt.client
		self._paho_mqtt = PahoMQTT.Client(clientID, False) 

		# register the callback
		self._paho_mqtt.on_connect = self.myOnConnect
		self._paho_mqtt.on_message = self.myOnMessageReceived


	def myOnConnect (self, paho_mqtt, userdata, flags, rc):
		print ("Connected to %s with result code: %d" % (self.broker, rc))

	def myOnMessageReceived (self, paho_mqtt , userdata, msg):
		# A new message is received
		self.notifier.notify (msg.topic, msg.payload)


	def myPublish (self, topic, msg):
		# if needed, you can do some computation or error-check before publishing
		print ("publishing '%s' with topic '%s'" % (msg, topic))
		# publish a message with a certain topic
		self._paho_mqtt.publish(topic, msg, 2)

	def mySubscribe (self, topic):
		# if needed, you can do some computation or error-check before subscribing
		print ("subscribing to %s" % (topic))
		# subscribe for a topic
		self._paho_mqtt.subscribe(topic, 2)

		# just to remember that it works also as a subscriber
		self._isSubscriber = True
		self._topic = topic

	def start(self):
		#manage connection to broker
		self._paho_mqtt.connect(self.broker , self.port)
		self._paho_mqtt.loop_start()

	def stop (self):
		if (self._isSubscriber):
			# remember to unsuscribe if it is working also as subscriber 
			self._paho_mqtt.unsubscribe(self._topic)

		self._paho_mqtt.loop_stop()
		self._paho_mqtt.disconnect()





In [6]:


class DoSomething():
	def __init__(self, clientID):
		# create an instance of MyMQTT class
		self.clientID = clientID
		self.myMqttClient = MyMQTT(self.clientID, "test.mosquitto.org", 1883, self)
		


	def run(self):
		# if needed, perform some other actions befor starting the mqtt communication
		print ("running %s" % (self.clientID))
		self.myMqttClient.start()

	def end(self):
		# if needed, perform some other actions befor ending the software
		print ("ending %s" % (self.clientID))
		self.myMqttClient.stop ()

	def notify(self, topic, msg):
		# manage here your received message. You can perform some error-check here  
		print ("received '%s' under topic '%s'" % (msg, topic))


In [7]:
import time


def get_Alert():
    test = DoSomething("subscriber 1")
    test.run()
    test.myMqttClient.mySubscribe('sdfsdf')
    test.end()


In [8]:
import requests
import sys  
sys.path.insert(0, './MQTT')
import subscriber

url = 'http://127.0.0.1:8080'

model_name = 'CNN.tflite'
tthres = 0.1
hthres = 0.2
url_predict = url + '/{}?model={}&tthres={}&hthres={}'.format('predict', model_name, tthres, hthres)
req = requests.get(url_predict)
if req.status_code == 200:
    get_Alert()
    body = req.json()
    print(body)
else:
    print('Error:', req.text)


Error: <!DOCTYPE html PUBLIC
"-//W3C//DTD XHTML 1.0 Transitional//EN"
"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html>
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"></meta>
    <title>500 Internal Server Error</title>
    <style type="text/css">
    #powered_by {
        margin-top: 20px;
        border-top: 2px solid black;
        font-style: italic;
    }

    #traceback {
        color: red;
    }
    </style>
</head>
    <body>
        <h2>500 Internal Server Error</h2>
        <p>The server encountered an unexpected condition which prevented it from fulfilling the request.</p>
        <pre id="traceback">Traceback (most recent call last):
  File "C:\Users\aligh\anaconda3\envs\tf\lib\site-packages\cherrypy\_cprequest.py", line 638, in respond
    self._do_respond(path_info)
  File "C:\Users\aligh\anaconda3\envs\tf\lib\site-packages\cherrypy\_cprequest.py", line 697, in _do_respond
    response.body = self.handler()
  File "C:\Users\